# GCL Tutorial

Neural Networks have grown to an extent that are unfeasible to train on singular workstations. An alternative to this is to use a cluster collaborating together. Engineers developing tensorflow have developed various compontnets that help make this easier without going into low-level networking between the machines. 

After setting up the module, the user would then just need to setup the number of nodes in the cluster to run the job. Even though this was aimed at distributed computing, if one node is specified this is only run on a single machine as desired. 

## Distributed Tensorflow

Any distributed job is run on a group of machines called a cluster. This cluster would generally have a _master_ node, _worker_ nodes and _parameter server_ nodes.

- __Worker__: Performs training iterations of the graphs. No additional responsibility. All the worker nodes work together to train the model.<br><br>

- __Master__: Perfroms the same computation as the Worker nodes but also has extra responsibilities. These may include, evaluating the graph during training, initialization, resetting crashed nodes and saving checkpoints.<br><br>

- __PS__: Contain all the variables for the graph. These are shared between all the workers and the master. This helps saving up memory and not having replicated variables. <br>

Within the machines the CPU and Master communicated using a line called Direct Memory Access (DMA). This is a fast communication within the device. Within the cluster the different worker, master and ps nodes communicate using a type of open-source remote procedure (gRPC) developed at Google. This line is a fast between-machine comminication line for fast parameter sharing. However, it still can be overloaded so gRPC errors must be avoided or taken care of.

## Replicated Training

In order to speed up training, the job is run across different machines. However, the manner in which this job is distributed makes a big difference. The possible approaches are:

- __In-Graph Replication__: Here a single graph is built in the parameter server and multiple copies of different parts of the graph are processed by different nodes. This is relatively simple to setup and most of the times used when considering multiple GPUs only. However, this method does not scale well to hundreds of machines.<br><br>

- __Between-Graph Replication__: Here a graph is built on each worker, the parameters' updates are calculated and updated in the parameter servers. This is relatively complex to setup. However, scales well and is the go-to method in distributed tensorflow. In this type of replication, variables are saved on parameter servers. More regarding this type of replication will be explained in the next section.<br>

These can be trained either __asyncronously__ or __synchronously__. In sychronous training the parameter update happens after all the workers have finished their respective loop whilst in asynchronous update happens continously with no waiting. The advantage of synchronous training is that it requires less epochs but more time per epoch.

## Between-Graph Replication

In between-graph replication, the same graph is replicated over all the workers and master nodes. The parameters will be stored on the parameter server and shared by the nodes as shown in _Figure 1_. One of the issues one must keep an eye on when using this type of replication is that the gRPC line is not overloaded. GRPC errors are relatively common especially when using few parameter servers and a large number of workers.

<p align="center"><img src="images/between_graph_replication.png" width="600"></p><br>

### Server and Cluster Spec

Since all the machines need to communicate to the master and to the parameter servers, a function called [`tf.train.Server`](https://www.tensorflow.org/api_docs/python/tf/train/Server) is used to manage all the nodes. In each node, the `tf.train.Server` function is run with the job processing cluster information as an argument as shown in _Figure 2_. This cluster information is specified by the [`tf.train.ClusterSpec`](https://www.tensorflow.org/api_docs/python/tf/train/ClusterSpec) which accepts addresses of each of the nodes and sets up an object that represents the cluster as a set of tasks, organised into different jobs. 

<p align="center"><img src="images/server_connections.png" width="600"></p><br>

These two functions are used as shown in the snippet below. In the actual user submitted task, these functions are used in a method called `dispatch` which is a boilerplate code used to dispatch the job onto the different nodes. This `dispatch` function is explained further on.

In [ ]:
cluster = tf.train.ClusterSpec({"worker": ["worker0.example.com:2222",
                                           "worker1.example.com:2222",
                                           "worker2.example.com:2222"],
                                "ps": ["ps0.example.com:2222",
                                       "ps1.example.com:2222"]})

server = tf.train.Server(cluster_spec,
                         job_name=job_name,
                         task_index=task_index)

Once all the server has been setup, one could then place the different parameters on the parameter servers and any other ops on the workers as shown below.

In [ ]:
with tf.device("/job:ps/task:0"):
    weights_1 = tf.Variable(...)
    biases_1 = tf.Variable(...)

with tf.device("/job:ps/task:1"):
    weights_2 = tf.Variable(...)
    biases_2 = tf.Variable(...)

with tf.device("/job:worker/task:7"):
    input, labels = ...
    layer_1 = tf.nn.relu(tf.matmul(input, weights_1) + biases_1)
    logits = tf.nn.relu(tf.matmul(layer_1, weights_2) + biases_2)
    # ...
    train_op = ...

### Assigning variables and Ops to different Nodes

When using between-graph replication the way parameters are placed on PS servers effects greatly. One could place each of the parameters manually as shown above. However, this may get tedious and takes up a considerable amount of time especially when graphs get larger in size. Hence, most of the time the [`tf.train.replica_device_setter`](https://www.tensorflow.org/api_docs/python/tf/train/replica_device_setter) function is used to distribute the parameters accross the parameter servers and any other ops on the workers. The default strategy of placing the parameters on the PS nodes of the `tf.train.replica_device_setter` is round robin. However, this is not necessarily the best method. Some large weights vectors might all end up on one PS node and all smaller biases might end up on another PS node which would make distribution of data unequal and might cause gRPC errors or slowing down of training. Other parameter distribution strategies include _tf.contrib.training.GreedyLoadBalancingStrategy_ which attempts to balance the parameters equally on different PS nodes. In the task, this `tf.train.replica_device_setter` is used as follow:

In [ ]:
with tf.device(tf.train.replica_device_setter(cluster=cluster_spec)):

    # Features and label tensors as read using filename queue
    features, labels = model.input_fn(
        train_files,
        ...
    )
...

### Fault Tolerence and Monitored Training Session (MTS)

One of the problems of using a cluster of machines is that probabilistically, it is common that one of the nodes would crash during the jobs especially if the job is very long. Hence, saving checkpoints when using distributed training is of paramount importance. Having a checkpoint, would allow the job to continue from a saved state without havign to start trianing from the very beginning.

Since fault tolerence is not very simple to implement, especially since the it is ideal that the job would carry on even if one worker crashes, the [`MonitoredTrainingSession`](https://www.tensorflow.org/api_docs/python/tf/train/MonitoredTrainingSession) function was provided to handle most of the fault tolerence for the user. This type of session has many hooks that allow it to perform various operations. At the start, the `MonitoredTrainingSession` initializes all variables, allocates a `Coordinator` and `Queue runner` for input data queues, saves checkpoints, restores from checkpoints if needed, handles any crashed worker, master or parameter server, saves summaries and more. These are performed using functions called _hooks_ which run at a specific point in time during the session. These include `CheckpointSaverHook` and the `SummarySaverHook`. However, the `MonitoredTrainingSession` also allows the possibility of adding more hooks such as `after_run` hook that allows other code to be run only after a `session.run` is completed. Such hook can help to validate the graph preformance.

Please note that for summaries to be saved by the `MonitoredTrainingSession`, these should be within the `SUMMARIES` tensorflow collection.

### Using Higher Level APIs

Using the _Tensorflow Core_ functions can at times be tedious especially for simple jobs. Hence, higher level APIs such as the __Estimator API__, __Keras Model API__ and more help when using distributed tensorflow and handle most of the nuts and bolts behind it themselves. This is ideal when having simple jobs that are common to many others. However, for more complex tests which require a certain amount of custimization using _Tensorflow Core_ is usually suggested. 

This above is just a basic overview of distributed training and between-graph replication training. More information regarding this section can be found [here](https://www.tensorflow.org/deploy/distributed). Furthermore, [this](https://youtu.be/la_M6bCV91M) presentation from the _Tensflow Dev Summit 2017_ is also worth a watch.


### Functions of Interest

Some functions that might be of interest include:

1. Using __`config=tf.ConfigProto(allow_soft_placement=True)`__ when defining the session would allow change of device if the device is not available without giving an error. For example, if `tf.device(/job:worker/task:7/gpu:10)` is used and only one GPU is available this would be automatically changed to run on `gpu:0` without giving an error. Similarly, if `tf.device(/job:worker/task:7/gpu:0)` is used and no GPU is available the op is instead changed to run on `cpu:0` of the same node instead.<br><br>

2. Using __`config=tf.ConfigProto(log_device_placement=True)`__ when defining the session would create log outputs of all the variables and op locations when defining the session when running the task. This is useful to make sure all the variables are on the desired device.<br><br>

3. Using __`config=tf.ConfigProto(gpu_options=tf.GPUOptions(per_process_gpu_memory_fraction=GPU_MEMORY_FRACTION))`__ when defining the session allows control of the memory used by the session process on the GPU. If this command is not used all the memory available of the GPU would be used by the process and hence multiple processes on the same GPU would not be possible.<br><br>

4. Using __`options=tf.RunOptions(trace_level=tf.RunOptions.FULL_TRACE)`__ and __`run_metadata=tf.RunMetadata()`__ when using the `session.run` method and saving the `run_metadata` with the file writer to tensorboard would provide memory and compute time information in the graph section in tensorboard. This is useful to analyse whether there are certain bottlenecks in the computation of the graph. Please note that where multithreading is involves, such as the input data queue, the compute and memory information here are not recorded.<br>

### Input Queues

Traditionally, the dataset would be loaded and then provided in the `feed_dict`. However, there are several problems with this approach. First, if the dataset is too big to fit in memory this would be problemetic. Furthermore, such approach is rather slow to process. A better approach is to have the data loader within the same graph as the neural network. 

A `TFRecord` binary file can be used to maximise access speed. This can be read in batches and then the [`tf.train.shuffle_batch`](https://www.tensorflow.org/api_docs/python/tf/train/shuffle_batch) function can be used to create a queue and shuffle the batches. A queue in such scenario is important. A queue would allow multiple threads to load into the queue and then the main graph would read from the other end of the queue. Please note that when using queues, the function `tf.train.start_queue_runners` need to be run __post__ initilizing the variables to start the input queues. More information regarding threading and queues can be found [here](https://www.tensorflow.org/programmers_guide/threading_and_queues).

# Installing Google Cloud SDK

No matter the type of machine processing data-on, Google Cloud SDK requires python 2.7 to operate. For Ubuntu and Mac this is not a problem since both CPU and GPU version of tensorflow are available for python 2.7. However, on Windows only the python 3.5 version is available. This would be problematics since the GC SDK only works with python 2.7. If using windows, one of the solutions to this is to use the _Ubuntu on Windows_ terminal and install the Ubuntu version there. 

Please note that in the Windows version, Tensorflow __only__ works on python 3.5. Having python 3.6 would __not__ work.

## Setting Up

1. [Download and Install](https://www.tensorflow.org/install/) Tensorflow for Python 2.7 through Anaconda or otherwise. This can be either the CPU only version or the CPU+GPU version.

2. [Download and Install](https://cloud.google.com/sdk/) the GC SDK for the machine in use.

3. Follow the provided [tutorial](https://cloud.google.com/deployment-manager/docs/step-by-step-guide/installation-and-setup) to connect the machine to the Google Cloud Project online. 



# Understanding the Census Income Dataset Cloud ML Tutorial

Using _between-graph_ replication is not as simple as when using _in-graph_ replication. First the job needs to be dispached onto the cluster before training is started. This tutorial follows the _tensorflow core_ version of the [census cloud tutorial](https://github.com/GoogleCloudPlatform/cloudml-samples/tree/master/census). The way this job is set up is not necessarily the best structure for very large models. However, it does give an understanding of core functions when using distributed tensorflow.

The core trainer of this tutorial is made up of two scripts. These are _task.py_ and _model.py_. The task script, sets up the job, handles training, saving and evaluation. On the other hand, the model script has the saved model to be used during training and the data generator to be used in the model. These two scripts are explianed in detail in the chapters below.


## Task Function

The task function handles everything related to training. The main flowchart of this is shown in _Figure 1_. A parser first creates variables with the arguments passed on to the script. These variables would control the type of training conducted. They include the location of the train and evaluation files, the output job directory, the number of steps to train the model, batch size learning rate and more. The dispatch function sets up the job in the cloud. The run function is then run on each of the worker or master node. The parameter servers do not have the run function run on them. These are only present to store variables. No training happens on these nodes.

<br><p align="center"><img src="images/census_flowchart.png" width="300"></p><br>

### Parser
The parser function is the function responsible of getting parameters passed in the terminal command and creating variables out of them. In the census example, the parameters used are shown below. These are either essential and marked as required, or non-essential and hence have a default value. 

- __train-files__: Location of the training set files. This can be in google cloud storage. (required)<br><br>
- __eval-files__: Location of the validation set files. This can be in google cloud storage. (required)<br><br>
- __job-dir__: Directory to save the output files. (required)<br><br>
- __train-steps__: Number of steps to train the network. Each step is a processed batch.<br><br>
- __eval-steps__: Number of steps to validate the network. Each step is a processed batch.<br><br>
- __train-batch-size__: The training batch size.<br><br>
- __eval-batch-size__: The validation batch size. This is usually equal to the train-batch-size.<br><br>
- __learning-rate__: The learning rate to be used to train the network.<br><br>
- __eval-frequency__: The number of checkpoints to wait for before performing validation.<br><br>
- __first-layer-size__: The size of the first layer of the FFN. This parameter is model dependent.<br><br>
- __num-layers__: The number of layers to be used in the FFN. This parameter is model dependent.<br><br>
- __scale-factor__: The scale by which the output size reduces. This parameter is model dependent.<br><br>
- __num-epochs__: The number of epochs to which train the network. This is ignored if steps is given.<br><br>
- __export-format__: The export format for the saved_model binary. <br><br>
- __verbosity__: The logging verboisty.<br>

Note: It would be wise to add a warning for any unknown arguments. This would help advise the user if one of the arguments may be misspelled and hence not used as supposed to.

### Dispatch Function
The aim of the dispatch function is to get the cluster information from the `TF_CONFIG` environment variable, load the `ClusterSpec`, setup the server and run the script on each of the nodes. This dispatch function is a boilerplate code. This is given below. 

In [ ]:
def dispatch(*args, **kwargs):
    """Parse TF_CONFIG to cluster_spec and call run() method
    TF_CONFIG environment variable is available when running using
    gcloud either locally or on cloud. It has all the information required
    to create a ClusterSpec which is important for running distributed code.
    """

    tf_config = os.environ.get('TF_CONFIG')

    # If TF_CONFIG is not available run local
    if not tf_config:
        return run(target='', cluster_spec=None, is_chief=True, *args, **kwargs)

    tf_config_json = json.loads(tf_config)

    cluster = tf_config_json.get('cluster')
    job_name = tf_config_json.get('task', {}).get('type')
    task_index = tf_config_json.get('task', {}).get('index')

    # If cluster information is empty run local
    if job_name is None or task_index is None:
        return run(target='', cluster_spec=None, is_chief=True, *args, **kwargs)

    cluster_spec = tf.train.ClusterSpec(cluster)
    server = tf.train.Server(cluster_spec,
                             job_name=job_name,
                             task_index=task_index)

    # Wait for incoming connections forever
    # Worker ships the graph to the ps server
    # The ps server manages the parameters of the model.
    #
    # See a detailed video on distributed TensorFlow
    # https://www.youtube.com/watch?v=la_M6bCV91M
    if job_name == 'ps':
        server.join()
        return
    elif job_name in ['master', 'worker']:
        return run(server.target, cluster_spec, is_chief=(job_name == 'master'),
                   *args, **kwargs)

The dispatch function, first loads the `TF_CONFIG` variable. This has all the information of the cluster being used. If this environment variable is not available, the job is run only on the local machine. If it is present, the json contents are loaded, and the cluster information together with the task information are extracted. The `cluster_spec` is setup using the cluster information in the `TF_CONFIG` variable and the `tf.train.ClusterSpec` function. This `cluster_spec` is then used in the `tf.train.Server`function to setup a server. Since this script is run on each node, including parameter servers and workers, if the machine running the script is a parameter server, it would then just run the `server.join()`, else the `run` function is executed.

### Run Function

The `run` function has various input parameters. Most of these are the ones specified in the parser. In addition to these, as can be seen in the dispatch function, are the `server.target` which is used to connect the training session to the server, `cluster_spec` which has the information of the cluster to be used by the `tf.train.replica_device_setter` when setting parameter save locations, and a boolean that specifies whether the respective node is master or not. 

In the `run` function, first the model layer sizes are calculated. This step is model dependent and is only performed if the same format is used. 

If the model is a chief, an evaluation graph is setup. Within this evaluation graph, a function that provides the features and labels of the validation data is specified together with a function of the network. This is similarly done for the training graph. However, in this case the training graph is built on both master and workers. For the training graph, the `tf.train.replica_device_setter` function is used to make decisions on where to save the parameters (local, ps, etc...).

To train the network a `MonitoredTrainingSession` is used. The benefits of this function is that it handles any crashes of the workers, parameter servers, or master, handles saving checkpoints and summaries, initialization of variables and additional hooks. 

In the `run` function, and `after_run` hook is added, which after each run, it counts the number of saved checkpoints from the previous validation, and if this is more than `eval-frequency` argument, it runs a validation session to gather metrics about the in-training graph. In the _Census Tutorial_, in the validation run, streaming metrics that process the whole epoch prior to releasing a metric value are used. In this validation script, the parameter values used in the checkpoint save file are loaded and used in the validation graph. This is not necessarily the best solution to validating whilst training. A second graph is needed, which would require more memory on the master machine. When the network is very large this might be a problem.
A different approach could be that the variables of the evaluation graph would be shared to the ones of the training graph. This would require a different setup than the one shown in the _Census Tutorial_.

The NN is training for the specified number of `train-steps` and validated for the specified number of `eval-steps`. After training, in the _Census Tutorial_ various test examples from three (CSV, JSON and TFRecord) input types are run to test the final performance of the network.

### Model

In the _Census Tutorial_ the model was a simple feed forward network. However, keeping the same structure, the network can be any type of neural network. As long as it accepts, input data and provides metrics, there are no restrictions of the shape of the model.

# Training using Command-line

Installation of Google cloud sdk and tensorflow

Command-line commands.